### 1.1 Cluster Mode
#### 一. Cluster Mode Overview
<img src="img/cluster-overview.png">
1. 每个spark application都有自己独立的executor来运行任务. 使得application之间彼此独立, 数据也不共享(除非写到外部存储系统上)  
2. Driver program需要从cluster manager上获取为它分配的excecuter的地址, 并在任务运行期间, 一直与其自身的executor保持连接  
3. spark不知道潜在的cluster manager实例是什么, 只要获取executor进行的链接即可. 因此, cluster manager可以很方便的在mesos,yarn,standalone之间选择

#### 二. Submitting Applications
[Submitting Applications](https://spark.apache.org/docs/latest/submitting-applications.html)
1. 一个参数  
`--deploy-mode`: cluster/client.   
  1. client模式: 把driver program作为外部client, 不在集群之中(默认)  
  2. cluster模式: 将driver program部署在集群中, 可以最小化driver和executor的延迟. 

### 1.2 Job Scheduling
#### 一. spark2个维度的调度
1. application间的作业调度 : 因为每个application都是独立的一组executor集合, 所以需要cluster manager协调资源  
2. applcation内的作业调度 : 每个application内部都会同时启动多个线程执行多个jobs, 因此需要` fair scheduler`调度这些jobs  

#### 二. Scheduling Across Applications  
1. Dynamic Resource Allocation  
jons的动态资源占用 : 在粗粒度cluster manager模式下, application可以动态调整自己占用的硬件资源. 即当不再使用资源时可将占据的资源还给集群, 并在需要的时候重新向cluster申请资源. 动态资源分配默认关闭, 在粗粒度模式下( standalone mode, YARN mode, and Mesos coarse-grained mode.)可以开启. mesos细粒度模式在spark2.0已被抛弃  
2. Resource Allocation Policy   
spark如何决定开启多少个executors来运行集群中的任务呢?   
spark采用启发式策略:  
  1. Request Policy  
   当有job被挂起时, 说明集群中缺少足够的executor来执行该任务, 就应该启动更多executor.通过配置`spark.dynamicAllocation.schedulerBacklogTimeout`来决定任务挂起后多少秒申请更多的executor.jon申请的executor个数是分轮次的, 每轮申请的个数都呈指数级增长 (1,2,4,8个executor的增长).采用指数级增长策略的原因有两个：  
     1. 对于任何一个Spark应用如果只是需要多申请少数几个执行器的话，那么必须非常谨慎地启动资源申请，这和TCP慢启动有些类似；  
     2. 第二，如果一旦Spark应用确实需要申请很多个执行器的话，那么可以确保其所需的计算资源及时地增长。
  2. Remove Policy  
   当一个executor长时间空闲时就会被停止. 通过`spark.dynamicAllocation.executorIdleTimeout`配置空闲时间
   
3. 如何关闭executor
  1. `非动态占用资源模式下`, executor在2种场景下会退出 : executor执行失败或执行的application已经退出. 此时executor的状态可以被安全丢弃   
  2. 在`动态资源占用模式`下, application将资源还给集群时就可以停止executor, 但该application仍在其它executor上执行着. 如果直接丢弃executor的执行状态, 会导致重新使用该executor计算结果时重算这部分数据.这点在shuffle阶段尤为明显. 比如负责map的executor把映射好的文件写在本地, 而后等待其它执行shuffle的executor向它取数据, 此时若负责map的executor已退出, 则这些写在磁盘上的文件就无法再被找到, 最后需要重算这部分数据.     
     要解决这一问题，就需要用到一个外部混洗服务（external shuffle service），该服务在Spark 1.2引入。该服务在集群中的每个节点上都会启动一个不依赖于任何Spark application或executor的独立进程。一旦该服务启用，executor不再从其它executor上获取shuffle文件，转而从这个service获取。这意味着，任何executor输出的混洗状态数据的存留时间比对应的executor进程还长。
     
#### 三. Scheduling Within an Application
1. application内部算子的资源调度  
  1. 默认，Spark应用内部使用FIFO调度策略。每个作业被划分为多个阶段（stage）（例如：map阶段和reduce阶段），第一个作业在其启动后会优先获取所有的可用资源，然后是第二个作业再申请，再第三个……。如果前面的作业没有把集群资源占满，则后续的作业可以立即启动运行，否则，后提交的作业会有明显的延迟等待。   
  2. 不过从Spark 0.8开始，Spark也能支持application内部各个job间的公平（Fair）调度。公平调度时，Spark以轮询的方式给每个作业分配资源，因此所有的作业获得的资源大体上是平均分配。这意味着，即使有大作业在运行，小的作业再提交也能立即获得计算资源而不是等待前面的作业结束，大大减少了延迟时间。这种模式特别适合于多用户配置。  
2. 公平调度资源池  
  1. 公平调度器还可以支持将作业分组放入资源池（pool），然后给每个资源池配置不同的选项（如：权重）。这样你就可以给一些比较重要的作业创建一个“高优先级”资源池，或者你也可以把每个用户的作业分到一组，这样一来就是各个用户平均分享集群资源，而不是各个作业平分集群资源。Spark公平调度的实现方式基本都是模仿 Hadoop Fair Scheduler 来实现的。  
  2. 默认情况下，新提交的作业都会进入到默认资源池中，不过作业对应于哪个资源池，可以在提交作业的线程中用SparkContext.setLocalProperty 设定 spark.scheduler.pool 属性。  
  3. 一旦设好了局部属性，所有该线程所提交的作业（即：在该线程中调用action算子，如：RDD.save/count/collect 等）都会使用这个资源池。这个设置是以线程为单位保存的，你很容易实现用同一线程来提交同一用户的所有作业到同一个资源池中。同样，如果需要清除资源池设置，只需在对应线程中调用如下代码：`sc.setLocalProperty("spark.scheduler.pool", null)`  
  4. 资源池默认行为  
     默认地，各个资源池之间平分整个集群的资源（包括default资源池），但在资源池内部，默认情况下，作业是FIFO顺序执行的。举例来说，如果你为每个用户创建了一个资源池，那么久意味着各个用户之间共享整个集群的资源，但每个用户自己提交的作业是按顺序执行的，而不会出现后提交的作业抢占前面作业的资源。



### 1.3 Spark on Yarn
#### 1. 指定`--master yarn`
#### 2. `--deploy-mode`分为cluster和client。
 cluster模式下， Driver程序运行在yarn管理的ApplicationMaster进程内  
 client模式下， Driver程序运行在客户端， ApplicationMaster仅用于申请资源
#### 3. 如何查看日志  
 1. yarn开启日志聚合时， 会将运行日志上传到hdfs。通过命令`yarn logs -applicationId <app ID>`查看。 也可以在spark的日志页面查看
 2. 未开启日志聚合， 需要到每台机器上查看container的运行日志
 
#### 4. 开启Spark history Server   
 webUI的4040端接口在Driver退出后就失效了， 因此需要使用Spark history Server记录历史日志。 默认端口在18080页面
 
#### 5. spark job使用log4j
  1. 在`SPARK_HOME/conf`下有日志配置， 来设置所有driver程序一致的日志输出级别  
  2. spark job独立配置的log4j的方法  
   要为应用主程序（即driver端）或执行程序使（即executor端）自定义log4j配置，需要两步就完成了，下面具体说明。  
   第一步：上传自定义 log4j-driver.properties和log4j-executor.properties     
   第二步：添加-Dlog4j的配置。使用 --conf参数。         
   用于驱动程序：`spark.driver.extraJavaOptions = -Dlog4j.configuration = <配置文件的位置>`        
   用于执行者：`spark.executor.extraJavaOptions= -Dlog4j.configuration = <配置文件的位置>`        
   注意：driver端还可以使用spark-submit的`--driver-java-options`参数去配置。    
   方案一：使用  spark-submit的 `--files` 参数将自定义的配置文件上传到应用程序的文件列表中。  
   `spark-submit --class com.hm.spark.Application 
         --master yarn 
         --deploy-mode cluster 
         --driver-cores 1 
         --driver-memory 1G 
         --num-executors 2 
         --executor-cores 1 
         --executor-memory 1G 
         --driver-java-options "-Dlog4j.configuration=log4j-driver.properties" 
         --conf spark.executor.extraJavaOptions="-Dlog4j.configuration=log4j-executor.properties" 
         --files /home/hadoop、log4j-driver.properties,/home/hadoop、log4j-executor.properties 
     /home/hadoop/spark-workspace/my-spark-etl-assembly-1.0-SNAPSHOT.jar`  
   注意，这里我没有使用`spark.driver.extraJavaOptions`参数去配置，而是使用spark-submit的`--driver-java-options`参数进行设置的。     
   方案二：不使用  spark-submit的 --files 参数上传文件，直接使用文件。    
`spark-submit --class com.hm.spark.Application 
        --master yarn 
        --deploy-mode cluster 
        --driver-cores 1 
        --driver-memory 1G 
        --num-executors 2 
        --executor-cores 1 
        --executor-memory 1G 
        --driver-java-options "-Dlog4j.configuration=file:/home/hadoop/log4j-driver.propertiess" 
        --conf spark.executor.extraJavaOptions="-Dlog4j.configuration=file:/home/hadoop//log4j-exec.properties" 
 /home/hadoop/spark-workspace/my-spark-etl-assembly-1.0-SNAPSHOT.jar
`
   注意：如果使用文件，  file: 则应明确提供配置文件的，并且文件需要在所有节点上本地存在。   
   
#### 6. 配置外部Shuffle Service
1. 拷贝`spark-<version>-yarn-shuffle.jar`文件到所有NodeManager的节点上  
2. 增加NodeManager的堆内存  
3. 修改`yarn-site。xml`配置项  
  yarn.nodemanager.aux-services.spark_shuffle.class = org.apache.spark.network.yarn.YarnShuffleService.  
4. spark中配置  
  spark.yarn.shuffle.stopOnFailure = false	

### 1.4 Spark On Mesos
#### 1. 提交mesos
```bash
./bin/spark-submit \
  --class org.apache.spark.examples.SparkPi \
  --master mesos://207.184.161.138:7077 \
  --deploy-mode cluster \
  --supervise \
  --executor-memory 20G \
  --total-executor-cores 100 \
  http://path/to/examples.jar \
  1000
```
#### 2. mesos的2种模式
1. Coarse-Grained  
优点： spark task启动快。 该模式下每个application占据一定的资源， 直到该应用结束  
2. Fine-Grained  
每个spark-task会在执行过程中动态申请/释放资源， 但spark task启动慢，不适用于低延时场景（现已废弃）